
### 1. **Objetivos**

**Objetivo do Modelo**:
- O objetivo deste modelo é prever dois fatores críticos para a produção:
  
  1. **Total de defeitos** esperados para uma nova ordem de produção, com base em medidas físicas (peso, comprimento e diâmetro).
  
  2. **Dias críticos de defeitos**, ou seja, prever em qual dia da produção será mais provável ocorrer o maior número de defeitos.

Essas previsões serão usadas para:
- Melhorar a **qualidade da produção**, permitindo ações preventivas em dias críticos.
- **Otimizar o processo de produção**, com base no histórico de ordens anteriores.
- **Ajudar o time de vendas e orçamento** a planejar e dimensionar ordens de forma mais eficiente, antecipando o comportamento de defeitos com base nas características do pedido e na duração da ordem.

### 2. **Solução**

**Solução Proposta**:
- Para prever o **total de defeitos**, usaremos um modelo de regressão, que será treinado com dados históricos sobre medidas físicas (peso, comprimento e diâmetro) e o total de defeitos observados em ordens anteriores.
- Para prever os **dias críticos de defeitos**, utilizaremos uma análise de sazonalidade, que considera o padrão de defeitos ao longo dos dias de produção de cada ordem. Isso nos ajudará a identificar quais dias (por exemplo, dia 2) têm maior chance de apresentar defeitos.
- Além disso, a **duração da ordem** (quantidade de dias de produção) será um parâmetro importante para ajustar as previsões. Ordens mais longas tendem a ter mais defeitos, e a duração influencia o comportamento dos defeitos ao longo dos dias.

**Modelo Utilizado**:
- **Regressão (ex: Gradient Boosting)**: Para prever o **total de defeitos** com base nas características do pedido e a duração da ordem.
- **Classificação ou análise de série temporal**: Para identificar **quais dias** terão maior probabilidade de defeitos ao longo da produção.

### 3. **Como o Modelo Vai Funcionar (Inputs e Outputs)**

#### Exemplo:
**Inputs (Entradas do Modelo)**:
- **Medidas da Ordem**:
  - Peso médio (g)
  - Comprimento médio (mm)
  - Diâmetro
- **Duração da Ordem**:
  - Número de dias que a produção vai durar (fornecido pelo time de vendas ou orçamento).
- **Dados Históricos**: O modelo é treinado com ordens passadas, suas medidas e os defeitos observados.

**Processamento**:
- O modelo será treinado com essas entradas para aprender os padrões que relacionam as **medidas físicas** e a **duração** com os **defeitos** observados em ordens passadas.
- O modelo ajustará as previsões com base nos padrões de **sazonalidade** dos defeitos, observando quais dias em ordens passadas têm mais defeitos.

**Outputs (Saídas do Modelo)**:
- **Previsão do Total de Defeitos**: Para uma nova ordem, o modelo preverá o total de defeitos, com base nas novas medidas fornecidas.
- **Previsão dos Dias Críticos de Defeitos**: O modelo também preverá os dias críticos de defeitos, ou seja, os dias específicos em que a ordem pode apresentar mais defeitos, com base no histórico de ordens anteriores e sua sazonalidade.

#### Exemplo de como isso pode ser explicado em termos simples:
- **Input**: Você fornecerá ao modelo as **medidas físicas** de uma nova ordem (peso, comprimento, diâmetro) e o **número de dias de produção** (duração).
- **Output**: O modelo retornará dois tipos de informações:
  
  1. **O total de defeitos esperados** para essa ordem.
  
  2. **Quais dias da ordem** terão maior probabilidade de defeitos, ajudando a identificar **o dia crítico** de defeitos.

### Exemplo de Diagrama ou Fluxograma:
Você também pode usar um fluxograma simples para ilustrar o processo de previsão do modelo, como por exemplo:

1. **Input**: Medidas da ordem (peso, comprimento, diâmetro, duração da ordem)

2. **Processamento**:
   - Treinamento com dados históricos (medidas + defeitos + duração + sazonalidade).
   - Ajuste de previsões com base na duração e no padrão de defeitos.

3. **Output**:
   - Previsão do total de defeitos.
   - Previsão dos dias críticos de defeitos.

### 4. **Conclusão**

- **Benefícios**:
  
  - **Previsões mais precisas** de defeitos, com base em dados reais e a duração de cada ordem.
  
  - **Ação preventiva**: Saber antecipadamente quais dias têm mais chance de defeitos pode otimizar os processos de produção.
  
  - **Apoio à decisão**: Ajudar os times de produção e vendas a planejar melhor as ordens e a controlar a qualidade da produção.

## Implementação do Modelo

### Importando bibliotecas

In [126]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

### Importando dataset

In [127]:
dados = pd.read_csv('../dados/medidas_geral_e_defeitos.csv')
df = pd.DataFrame(dados)

In [128]:
df = df.set_index(['Ordem de prod', 'Ref. do Artigo', 'Data de prod'])

In [129]:
df

Temperatura (F°)  Peso médio (g)  \
Ordem de prod Ref. do Artigo Data de prod                                     
192970        C0516.0000R    2024-01-03             2126.97           169.3   
                             2024-01-04             2118.92           169.7   
                             2024-01-05             2125.00           173.6   
                             2024-01-06             2125.40           170.2   
                             2024-01-07             2126.76           171.7   
...                                                     ...             ...   
194121        C0516.0000R    2024-06-26             2154.44           169.0   
                             2024-06-27             2152.32           168.9   
                             2024-06-28             2145.60           169.3   
                             2024-06-29             2145.60           168.8   
                             2024-06-30             2148.00           169.2   

                                           Comprimento médio (mm)  Diâmetro  \
Ordem de prod Ref. do Artigo Data de prod                                     
192970        C0516.0000R    2024-01-03                     110.4      32.5   
                             2024-01-04                      85.6      18.1   
                             2024-01-05                     140.5      32.9   
                             2024-01-06                     144.7      32.7   
                             2024-01-07                     141.5      30.2   
...                                                           ...       ...   
194121        C0516.0000R    2024-06-26                     124.3      32.6   
                             2024-06-27                     120.2      33.6   
                             2024-06-28                     124.1      34.2   
                             2024-06-29                     120.0      33.5   
                             2024-06-30                     122.9      34.6   

                                           Total de Defeitos no Dia  \
Ordem de prod Ref. do Artigo Data de prod                             
192970        C0516.0000R    2024-01-03                         222   
                             2024-01-04                         249   
                             2024-01-05                         268   
                             2024-01-06                         242   
                             2024-01-07                         232   
...                                                             ...   
194121        C0516.0000R    2024-06-26                         258   
                             2024-06-27                         218   
                             2024-06-28                         218   
                             2024-06-29                         213   
                             2024-06-30                         256   

                                          Defeito Mais Frequente do Dia  \
Ordem de prod Ref. do Artigo Data de prod                                 
192970        C0516.0000R    2024-01-03                    Sujo de óleo   
                             2024-01-04                    Sujo de óleo   
                             2024-01-05                    Sujo de óleo   
                             2024-01-06                    Sujo de óleo   
                             2024-01-07                    Sujo de óleo   
...                                                                 ...   
194121        C0516.0000R    2024-06-26                    Sujo de óleo   
                             2024-06-27                    Sujo de óleo   
                             2024-06-28                    Sujo de óleo   
                             2024-06-29                    Sujo de óleo   
                             2024-06-30                    Sujo de óleo   

                                           Sujo de óleo  Rugas  \
Ordem de prod Ref. do

### Verificando se o dataframe está limpo e padronizado

In [130]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 200 entries, (192970, 'C0516.0000R', '2024-01-03') to (194121, 'C0516.0000R', '2024-06-30')
Columns: 133 entries, Temperatura (F°) to Crú
dtypes: float64(4), int64(128), object(1)
memory usage: 216.2+ KB


In [131]:
df.isnull().sum()

Temperatura (F°)            0
Peso médio (g)              0
Comprimento médio (mm)      0
Diâmetro                    0
Total de Defeitos no Dia    0
                           ..
Ferrugem no corpo           0
Fagulha                     0
Alça deformada              0
Instabilidade               0
Crú                         0
Length: 133, dtype: int64

In [132]:
df_reset = df.reset_index()

In [133]:
df_reset

,Ordem de prod,Ref. do Artigo,Data de prod,Temperatura (F°),Peso médio (g),Comprimento médio (mm),Diâmetro,Total de Defeitos no Dia,Defeito Mais Frequente do Dia,Sujo de óleo,...,Bolha aberta interna,Cascão cortante,Espessura abaixo do mínimo,Fundo deformado,Bolha na alça,Ferrugem no corpo,Fagulha,Alça deformada,Instabilidade,Crú
0,192970,C0516.0000R,2024-01-03,2126.97,169.3,110.4,32.5,222,Sujo de óleo,31,...,0,0,0,0,0,0,0,0,0,0
1,192970,C0516.0000R,2024-01-04,2118.92,169.7,85.6,18.1,249,Sujo de óleo,49,...,0,0,0,0,0,0,0,0,0,0
2,192970,C0516.0000R,2024-01-05,2125.00,173.6,140.5,32.9,268,Sujo de óleo,49,...,0,0,0,0,0,0,0,0,0,0
3,192970,C0516.0000R,2024-01-06,2125.40,170.2,144.7,32.7,242,Sujo de óleo,73,...,0,0,0,0,0,0,0,0,0,0
4,192970,C0516.0000R,2024-01-07,2126.76,171.7,141.5,30.2,232,Sujo de óleo,49,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,194121,C0516.0000R,2024-06-26,2154.44,169.0,124.3,32.6,258,Sujo de óleo,40,...,1,0,0,0,0,0,0,0,0,0
196,194121,C0516.0000R,2024-06-27,2152.32,168.9,120.2,33.6,218,Sujo de óleo,75,...,0,0,0,0,0,0,0,0,0,0
197,194121,C0516.0000R,2024-06-28,2145.60,169.3,124.1,34.2,218,Sujo de óleo,54,...,0,0,0,0,0,0,0,0,0,0
198,194121,C0516.0000R,2024-06-29,2145.60,168.8,120.0,33.5,213,Sujo de óleo,54,...,0,0,0,0,0,0,1,0,0,0


# Solução - Prever total de defeitos e defeitos por dia

## 1. Complementar a previsão de defeitos
O modelo pode ser alimentado não apenas pelas características do pedido (medidas), mas também pela duração da ordem para calcular mais precisamente o total de defeitos.
Depoi de analisar, ordens mais longas tendem a ter mais defeitos. Portanto, a duração poderia atuar como um fator adicional importante, ajudando o modelo a ajustar sua previsão de defeitos com base na duração real da ordem.

## 2. Prever o total de defeitos, defeitos por dia e dia crítico 
Com a duração da ordem fornecida, podemos ajustar a sazonalidade dos defeitos. A ideia seria usar a duração para:

Mapear quais dias da ordem têm maior probabilidade de defeitos, mesmo antes da produção começar. Ou seja, o modelo pode indicar se a ordem tende a ter mais defeitos nos primeiros dias ou se a distribuição dos defeitos será mais uniforme ao longo da produção.
Com ordens de duração maior, a previsão de defeitos poderia ser ajustada para mostrar que mais defeitos podem ocorrer nos dias intermediários ou finais.

### Padrões Cíclicos: 
Se, por exemplo, o Dia 2 costuma ter um número elevado de defeitos, isso pode sugerir problemas recorrentes ou características intrínsecas do processo nesse estágio.

### Comparação Escalável: 
Como as ordens têm durações diferentes, é difícil compará-las diretamente. Normalizar os dias cria um padrão comum que facilita a análise.

### Previsibilidade: 
A partir dessa análise, você pode prever que, em uma nova ordem, o Dia 2 (ou qualquer outro dia normalizado) tem maior probabilidade de apresentar mais defeitos.

## 3. Modelagem ajustada pela duração
Criar uma lógica que leva em consideração a duração da ordem:

**Entradas**: Receberia não apenas as medidas físicas (peso, comprimento, diâmetro) e a duração da ordem, mas também a data de início ou a quantidade de dias de produção.

**Processamento**: Com a duração, o modelo poderia ajustar a previsão de defeitos, utilizando o padrão de sazonalidade (dias de maior probabilidade de defeitos).

**Saída**: O modelo retornaria tanto o total de defeitos estimados quanto os dias críticos de defeitos (e.g., o segundo dia da ordem, conforme você observou, com maior chance de defeitos).


In [137]:
df_sazonalidade = df_reset.groupby(['Ordem de prod', 'Data de prod'])[['Total de Defeitos no Dia']].sum().reset_index()

In [138]:
df_sazonalidade['Data de prod'] = pd.to_datetime(df_sazonalidade['Data de prod'])

In [139]:
df_sazonalidade['Dia da Semana'] = df_sazonalidade['Data de prod'].dt.day_name()
df_sazonalidade['Mês'] = df_sazonalidade['Data de prod'].dt.month
df_sazonalidade['Dia do Ano'] = df_sazonalidade['Data de prod'].dt.dayofyear

In [140]:
duracao_por_ordem = df_sazonalidade.groupby('Ordem de prod')['Data de prod'].nunique().reset_index()
duracao_por_ordem.columns = ['Ordem de prod', 'Dias de Duração']

duracao_por_ordem

,Ordem de prod,Dias de Duração
0,192970,9
1,193112,3
2,193164,12
3,193206,4
4,193207,10
5,193208,2
6,193209,7
7,193342,20
8,193368,7
9,193459,3


In [167]:
defeitos_por_duracao = df_sazonalidade.groupby('Ordem de prod').agg({'Total de Defeitos no Dia': 'sum', 'Dias de Duração': 'first'})

In [168]:
defeitos_por_duracao = defeitos_por_duracao.rename(columns={'Total de Defeitos no Dia':'Total de Defeitos por Ordem'})

In [169]:
px.scatter(data_frame=defeitos_por_duracao, 
           x='Dias de Duração', 
           y='Total de Defeitos por Ordem', 
           color='Total de Defeitos por Ordem', 
           color_continuous_scale='peach', 
           size='Total de Defeitos por Ordem')

In [141]:
df_sazonalidade = df_sazonalidade.merge(duracao_por_ordem, on='Ordem de prod', how='left')

In [142]:
df_sazonalidade

,Ordem de prod,Data de prod,Total de Defeitos no Dia,Dia da Semana,Mês,Dia do Ano,Dias de Duração
0,192970,2024-01-03,222,Wednesday,1,3,9
1,192970,2024-01-04,249,Thursday,1,4,9
2,192970,2024-01-05,268,Friday,1,5,9
3,192970,2024-01-06,242,Saturday,1,6,9
4,192970,2024-01-07,232,Sunday,1,7,9
...,...,...,...,...,...,...,...
195,194121,2024-06-26,258,Wednesday,6,178,6
196,194121,2024-06-27,218,Thursday,6,179,6
197,194121,2024-06-28,218,Friday,6,180,6
198,194121,2024-06-29,213,Saturday,6,181,6


## Média de defeitos por dias da semana

In [143]:
df_medias_dia_semana = df_sazonalidade.groupby('Dia da Semana')[['Total de Defeitos no Dia']].mean().sort_values('Total de Defeitos no Dia', ascending=False).round(2)

In [144]:
fig = px.bar(df_medias_dia_semana.reset_index(), 
             x='Dia da Semana', y='Total de Defeitos no Dia',
             text='Total de Defeitos no Dia')

fig.update_traces(textposition='outside', texttemplate='%{text}', 
                  textfont_size=14, textfont_color='black')

fig.show()

## Média de defeitos por dia de produção

In [194]:
defeitos_por_dia_normalizado = df_sazonalidade.groupby(['Dia Normalizado'])[['Total de Defeitos no Dia']].mean()
defeitos_por_dia_normalizado = defeitos_por_dia_normalizado.reset_index()
defeitos_por_dia_normalizado.columns = ['Dia Normalizado', 'Média de Defeitos']

fig = go.Figure()

fig.add_trace(go.Scatter(x=defeitos_por_dia_normalizado['Dia Normalizado'],
                         y=defeitos_por_dia_normalizado['Média de Defeitos'],
                         mode='lines+markers', 
                         marker=dict(size=10),
                         name='Média de Defeitos'))

fig.update_layout(
    title='Média de Defeitos por Dia Normalizado',
    title_font_size=16,
    xaxis_title='Dia Normalizado',
    xaxis_title_font_size=14,
    yaxis_title='Média de Defeitos',
    yaxis_title_font_size=14,
    template='plotly_dark',
)

fig.show()

## Implementação do Modelo de Regressão

In [150]:
df_total_duracao_reset = defeitos_por_duracao.reset_index()

In [174]:
# Passo 1: Preparar os dados para o total de defeitos e a duração da ordem
df_combined = df_reset.merge(df_total_duracao_reset[['Ordem de prod', 'Total de Defeitos por Ordem', 'Dias de Duração']], on='Ordem de prod', how='left')

# Passo 2: Separar variáveis independentes (medidas e duração) e dependente (total de defeitos)
X = df_combined[['Peso médio (g)', 'Comprimento médio (mm)', 'Diâmetro', 'Dias de Duração']]  # Incluindo a duração
y = df_combined['Total de Defeitos por Ordem']

# Passo 3: Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Passo 4: Normalizar os dados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Passo 5: Treinar o modelo para prever o total de defeitos
gb_model = GradientBoostingRegressor(random_state=42)
gb_model.fit(X_train_scaled, y_train)

# Passo 6: Previsões de total de defeitos
y_pred_gb = gb_model.predict(X_test_scaled)

# Passo 7: Avaliar a performance do modelo
mse_gb = mean_squared_error(y_test, y_pred_gb)
r2_gb = r2_score(y_test, y_pred_gb)

print(f"Gradient Boosting - Mean Squared Error (MSE): {mse_gb}")
print(f"Gradient Boosting - R² Score: {r2_gb}")

Gradient Boosting - Mean Squared Error (MSE): 7698.757546270441
Gradient Boosting - R² Score: 0.9918282458806885


### Testando modelo

In [188]:
nova_ordem = {
    'Peso médio (g)': 168.5,
    'Comprimento médio (mm)': 29.3,
    'Diâmetro': 15.9,
    'Dias de Duração': 8 
}

X_nova_ordem = scaler.transform(pd.DataFrame([nova_ordem])) 
total_defeitos_previsto = gb_model.predict(X_nova_ordem)
print(f"Total de defeitos previsto para a nova ordem: {total_defeitos_previsto[0]}")

Total de defeitos previsto para a nova ordem: 1780.4143301480754


In [201]:
# Passo 2: Calcular a distribuição dos defeitos da nova ordem
# Ajustando a distribuição proporcionalmente ao total de defeitos previsto
# Aqui, total_defeitos_previsto deve ser um número escalar, sem necessidade de indexação
defeitos_por_dia_normalizado['Defeitos Previstos'] = defeitos_por_dia_normalizado['Média de Defeitos'] / defeitos_por_dia_normalizado['Média de Defeitos'].sum() * total_defeitos_previsto

# Passo 3: Identificar o dia mais crítico com base na previsão de defeitos
dia_critico_previsto = defeitos_por_dia_normalizado.loc[defeitos_por_dia_normalizado['Defeitos Previstos'].idxmax()]

# Texto do dia crítico
print(f"Para esta ordem, o dia mais crítico seria o Dia {dia_critico_previsto['Dia Normalizado']} com {dia_critico_previsto['Defeitos Previstos'].round(2)} defeitos.")

# Passo 4: Ajustar a distribuição de defeitos para o número de dias de duração da nova ordem
dias_duracao = nova_ordem['Dias de Duração']

# Recalcular a distribuição dos defeitos considerando o número de dias de duração
defeitos_por_dia_normalizado['Defeitos Previstos'] = (defeitos_por_dia_normalizado['Média de Defeitos'] / defeitos_por_dia_normalizado['Média de Defeitos'].sum()) * total_defeitos_previsto

# Passo 5: Limitar a previsão de defeitos aos primeiros "dias_duracao" dias
df_dias_duracao = defeitos_por_dia_normalizado.head(dias_duracao)

# Passo 6: Criar o gráfico de linha com os defeitos previstos para os dias de duração
fig = px.line(df_dias_duracao, 
              x='Dia Normalizado', 
              y='Defeitos Previstos', 
              title=f'Previsão de Defeitos ao Longo dos {dias_duracao} Dias da Nova Ordem', 
              labels={'Dia Normalizado': 'Dia de Produção', 'Defeitos Previstos': 'Defeitos Previstos'})

# Adicionando marcador nos pontos
fig.update_traces(mode='lines+markers', line=dict(color='blue'))

# Exibindo o gráfico
fig.show()

# Passo 9: Exibir os defeitos previstos para os dias de duração
df_dias_duracao[['Dia Normalizado', 'Defeitos Previstos']].set_index('Dia Normalizado')

Para esta ordem, o dia mais crítico seria o Dia 2.0 com 112.55 defeitos.


,Defeitos Previstos
Dia Normalizado,
1,102.330978
2,112.548049
3,98.546473
4,101.213555
5,98.713954
6,97.912313
7,85.971422
8,92.087196
